In [1]:
# Import 3rd party libraries
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot as plt

# Import local libraries
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.base import clone
from sklearn.metrics import mean_squared_error

# Configure Notebook
%matplotlib inline
plt.style.use('fivethirtyeight')
sns.set_context("notebook")
import warnings
warnings.filterwarnings('ignore')

Read the data file.

In [2]:
data = pd.read_pickle('df_LA_parcels_LAcity_cleaned_REV2_MB.pkl', compression = "gzip")
data = data[data['TotalValue'] <= 2000000]
data = data[data['Neighborhood'].str.contains('Griffith Park') == False] # This is an outlier

In [3]:
data.shape

(417991, 20)

In [4]:
crime_data = pd.read_csv('neighbourhoods_crime_count_2010_to_2019.csv')
crime_data = crime_data[['name', 'crime_count']].rename(columns = {'name': 'Neighborhood'})
data = data.merge(crime_data, on = 'Neighborhood')
data

,AIN,SQFTmain,Bedrooms,Bathrooms,LandValue,LandBaseYear,TotalValue,YearBuilt,EffectiveYearBuilt,PropertyUseCode,...,Cluster,ZIPcode5,ZIPcode4,BusBenchClosestDist,SubwayStopClosestDist,Neighborhood,ShapeSTAre,ShapeSTLen,geometry,crime_count
0,2012031006,1650.0,4,2,236718.0,2011,368094.0,1963,1963,0100,...,2143,91304,3821,627.902037,81059.153781,Canoga Park,8115.922852,377.443437,"POLYGON ((-118.60779 34.21852, -118.60814 34.2...",4405
1,2012002007,1327.0,3,2,135211.0,1989,338036.0,1957,1957,0100,...,2143,91304,3838,199.820206,81531.724666,Canoga Park,8399.689453,379.990318,"POLYGON ((-118.60960 34.21914, -118.60983 34.2...",4405
2,2023013004,714.0,2,1,246192.0,2003,307598.0,1953,1953,0100,...,2143,91303,1112,1006.955311,77072.886615,Canoga Park,6749.179688,369.982730,"POLYGON ((-118.60956 34.20548, -118.60956 34.2...",4405
3,2023025020,1657.0,3,2,212535.0,1989,279800.0,1956,1966,0100,...,2143,91303,1011,1085.125000,77005.109741,Canoga Park,7629.740234,358.002982,"POLYGON ((-118.61205 34.20399, -118.61204 34.2...",4405
4,2023004004,1876.0,6,2,285871.0,1990,407892.0,1942,1947,0101,...,2143,91303,1110,634.633252,76729.509756,Canoga Park,13498.103516,469.973496,"POLYGON ((-118.60738 34.20548, -118.60721 34.2...",4405
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417986,7559018006,2187.0,4,3,654147.0,2018,830954.0,1955,1963,0100,...,14190,90732,3120,1388.764601,37850.477861,San Pedro,6029.481445,322.234960,"POLYGON ((-118.31383 33.74287, -118.31383 33.7...",8957
417987,7559008011,1305.0,3,2,609000.0,2007,762000.0,1955,1955,0100,...,14190,90732,2713,1882.772648,38133.074377,San Pedro,8677.246094,402.365689,"POLYGON ((-118.31531 33.74506, -118.31552 33.7...",8957
417988,7562022007,1910.0,3,2,51162.0,1975,124294.0,1961,1961,0100,...,14190,90732,4119,648.733626,42062.411545,San Pedro,6747.458984,334.916599,"POLYGON ((-118.32218 33.72681, -118.32230 33.7...",8957
417989,7563021007,4541.0,4,4,492258.0,2013,1251407.0,2007,2007,0100,...,14190,90732,4720,1809.655573,42164.699128,San Pedro,6000.456055,320.017959,"POLYGON ((-118.31985 33.72131, -118.31996 33.7...",8957


In [5]:
neighborhood_cluster_data = pd.read_pickle('neighborhoods_cluster.pkl')
data = data.merge(neighborhood_cluster_data[['Neighborhood', 'cluster']], on = 'Neighborhood')
data

,AIN,SQFTmain,Bedrooms,Bathrooms,LandValue,LandBaseYear,TotalValue,YearBuilt,EffectiveYearBuilt,PropertyUseCode,...,ZIPcode5,ZIPcode4,BusBenchClosestDist,SubwayStopClosestDist,Neighborhood,ShapeSTAre,ShapeSTLen,geometry,crime_count,cluster
0,2012031006,1650.0,4,2,236718.0,2011,368094.0,1963,1963,0100,...,91304,3821,627.902037,81059.153781,Canoga Park,8115.922852,377.443437,"POLYGON ((-118.60779 34.21852, -118.60814 34.2...",4405,4
1,2012002007,1327.0,3,2,135211.0,1989,338036.0,1957,1957,0100,...,91304,3838,199.820206,81531.724666,Canoga Park,8399.689453,379.990318,"POLYGON ((-118.60960 34.21914, -118.60983 34.2...",4405,4
2,2023013004,714.0,2,1,246192.0,2003,307598.0,1953,1953,0100,...,91303,1112,1006.955311,77072.886615,Canoga Park,6749.179688,369.982730,"POLYGON ((-118.60956 34.20548, -118.60956 34.2...",4405,4
3,2023025020,1657.0,3,2,212535.0,1989,279800.0,1956,1966,0100,...,91303,1011,1085.125000,77005.109741,Canoga Park,7629.740234,358.002982,"POLYGON ((-118.61205 34.20399, -118.61204 34.2...",4405,4
4,2023004004,1876.0,6,2,285871.0,1990,407892.0,1942,1947,0101,...,91303,1110,634.633252,76729.509756,Canoga Park,13498.103516,469.973496,"POLYGON ((-118.60738 34.20548, -118.60721 34.2...",4405,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417986,7559018006,2187.0,4,3,654147.0,2018,830954.0,1955,1963,0100,...,90732,3120,1388.764601,37850.477861,San Pedro,6029.481445,322.234960,"POLYGON ((-118.31383 33.74287, -118.31383 33.7...",8957,4
417987,7559008011,1305.0,3,2,609000.0,2007,762000.0,1955,1955,0100,...,90732,2713,1882.772648,38133.074377,San Pedro,8677.246094,402.365689,"POLYGON ((-118.31531 33.74506, -118.31552 33.7...",8957,4
417988,7562022007,1910.0,3,2,51162.0,1975,124294.0,1961,1961,0100,...,90732,4119,648.733626,42062.411545,San Pedro,6747.458984,334.916599,"POLYGON ((-118.32218 33.72681, -118.32230 33.7...",8957,4
417989,7563021007,4541.0,4,4,492258.0,2013,1251407.0,2007,2007,0100,...,90732,4720,1809.655573,42164.699128,San Pedro,6000.456055,320.017959,"POLYGON ((-118.31985 33.72131, -118.31996 33.7...",8957,4


Check out the correlation between each of the current features of the data set.

In [6]:
correlation = data.corr()

In [7]:
correlation

,AIN,SQFTmain,Bedrooms,Bathrooms,LandValue,LandBaseYear,TotalValue,YearBuilt,EffectiveYearBuilt,TaxRateArea,Cluster,ZIPcode5,BusBenchClosestDist,SubwayStopClosestDist,ShapeSTAre,ShapeSTLen,crime_count
AIN,1.000000,-0.179812,-0.225205,-0.233227,0.002198,-0.054774,-0.066662,-0.418312,-0.365073,0.008864,0.830914,-0.825312,-0.032332,-0.614568,-0.203228,-0.226468,-0.223031
SQFTmain,-0.179812,1.000000,0.647474,0.819407,0.307814,-0.014653,0.500990,0.394750,0.484244,-0.047995,-0.172559,0.132974,0.259634,0.188972,0.363467,0.419733,-0.088974
Bedrooms,-0.225205,0.647474,1.000000,0.651821,0.126503,0.009686,0.258229,0.347847,0.434352,-0.021023,-0.191702,0.224629,0.121317,0.292311,0.200032,0.235687,0.039100
Bathrooms,-0.233227,0.819407,0.651821,1.000000,0.293537,0.036667,0.469741,0.480377,0.599910,-0.048935,-0.221764,0.199417,0.240393,0.238472,0.287968,0.329468,-0.065150
LandValue,0.002198,0.307814,0.126503,0.293537,1.000000,0.618352,0.949364,0.087236,0.180084,-0.046918,-0.011803,-0.090732,0.147399,-0.080698,0.121652,0.138493,-0.215423
LandBaseYear,-0.054774,-0.014653,0.009686,0.036667,0.618352,1.000000,0.574064,0.063101,0.120769,0.016739,-0.053266,0.063693,-0.015329,0.047280,-0.027339,-0.034948,0.048150
TotalValue,-0.066662,0.500990,0.258229,0.469741,0.949364,0.574064,1.000000,0.215509,0.326938,-0.052627,-0.075075,-0.020359,0.199868,-0.010853,0.190470,0.216579,-0.199606
YearBuilt,-0.418312,0.394750,0.347847,0.480377,0.087236,0.063101,0.215509,1.000000,0.854818,-0.024475,-0.381231,0.473667,0.232888,0.506799,0.188496,0.204839,0.090084
EffectiveYearBuilt,-0.365073,0.484244,0.434352,0.599910,0.180084,0.120769,0.326938,0.854818,1.000000,-0.028758,-0.328556,0.401991,0.209069,0.409613,0.178307,0.198818,0.046346
TaxRateArea,0.008864,-0.047995,-0.021023,-0.048935,-0.046918,0.016739,-0.052627,-0.024475,-0.028758,1.000000,0.041323,0.021483,-0.039102,0.047516,-0.026556,-0.027413,0.106985


Look at the info of each column in the data set.

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 417991 entries, 0 to 417990
Data columns (total 22 columns):
 #   Column                 Non-Null Count   Dtype   
---  ------                 --------------   -----   
 0   AIN                    417991 non-null  int64   
 1   SQFTmain               417991 non-null  float64 
 2   Bedrooms               417991 non-null  int64   
 3   Bathrooms              417991 non-null  int64   
 4   LandValue              417991 non-null  float64 
 5   LandBaseYear           417991 non-null  int64   
 6   TotalValue             417991 non-null  float64 
 7   YearBuilt              417991 non-null  int64   
 8   EffectiveYearBuilt     417991 non-null  int64   
 9   PropertyUseCode        417991 non-null  object  
 10  TaxRateArea            417991 non-null  int64   
 11  Cluster                417991 non-null  int64   
 12  ZIPcode5               417991 non-null  int64   
 13  ZIPcode4               416208 non-null  object  
 14  BusBenchClosestDist 

In [9]:
data.isnull().sum()

AIN                         0
SQFTmain                    0
Bedrooms                    0
Bathrooms                   0
LandValue                   0
LandBaseYear                0
TotalValue                  0
YearBuilt                   0
EffectiveYearBuilt          0
PropertyUseCode             0
TaxRateArea                 0
Cluster                     0
ZIPcode5                    0
ZIPcode4                 1783
BusBenchClosestDist         0
SubwayStopClosestDist       0
Neighborhood                0
ShapeSTAre                  0
ShapeSTLen                  0
geometry                    0
crime_count                 0
cluster                     0
dtype: int64

Label the data into groups based on TotalValue, split into 4 groups. This is for stratifying the dataset when splitting into train, test, val sets so that the data split is balanced.

In [10]:
# Group the data into Quartiles based on TotalValue for stratifying purposes
data['Quartile_Number'] = 0

data.loc[
    (data['TotalValue'] >= 0) & 
    (data['TotalValue'] < data['TotalValue'].quantile(0.25)), 
    'Quartile_Number'] = 1

data.loc[
    (data['TotalValue'] >= data['TotalValue'].quantile(0.25)) & 
    (data['TotalValue'] < data['TotalValue'].quantile(0.5)), 
    'Quartile_Number'] = 2

data.loc[
    (data['TotalValue'] >= data['TotalValue'].quantile(0.5)) & 
    (data['TotalValue'] < data['TotalValue'].quantile(0.75)), 
    'Quartile_Number'] = 3

data.loc[
    (data['TotalValue'] >= data['TotalValue'].quantile(0.75)), 
    'Quartile_Number'] = 4

Split the data into training set, validation set, and test set.

In [11]:
train, evaluate = train_test_split(data, test_size = 0.3, random_state = 0, stratify = data['Quartile_Number'])
val, test = train_test_split(evaluate, test_size = 0.5, random_state = 0, stratify = evaluate['Quartile_Number'])

# Print results
print('Train {}%'.format(train.shape[0] / data.shape[0] * 100))
print('Val {}%'.format(val.shape[0] / data.shape[0] * 100))
print('Test {}%'.format(test.shape[0] / data.shape[0] * 100))

Train 69.9998325322794%
Val 15.000083733860297%
Test 15.000083733860297%


In [12]:
def select_columns(data, *columns):
    """Select only columns passed as arguments."""
    return data.loc[:, columns]

## Gradient Descent

    columns = [
     'SQFTmain',
     'Bedrooms',
     'Bathrooms',
     'LandBaseYear',
     'YearBuilt',
     'EffectiveYearBuilt',
     'ShapeSTAre',
     'ShapeSTLen',
     'crime_count',
     'YearBuilt_median',
     'EffectiveYearBuilt_median',
     'SQFTmain_median'
    ]
Columns to standardize

In [13]:
# Standardize features using standard scaling
def standardize_data(data, columns):
    """
    Standardize all the data to be number of std away from the mean.
    """
    
    for column in columns:
        data[column] = (data[column] - data[column].mean())/data[column].std()
    
    return data

### Try with Four Features

In [14]:
def process_data_GD(data):
    """Combine all pipelines to create processed data."""
    # Transform Data, Select Features
    data = select_columns(data, 
                         'SQFTmain',
                         'Bathrooms',
                         'LandBaseYear',
                         'TotalValue',
                         'EffectiveYearBuilt'
                         )
    
    # Standardize Data using Standard Scaling
    columns = [
     'SQFTmain',
     'Bathrooms',
     'LandBaseYear',
     'EffectiveYearBuilt'
    ]
    
    data = standardize_data(data, columns)
    
    # Return predictors and response variables separately
    X = data.drop(['TotalValue'], axis = 1)
    y = data.loc[:, 'TotalValue']
    
    return X, y

In [15]:
X_train, y_train = process_data_GD(train)
    
X_train.head()

,SQFTmain,Bathrooms,LandBaseYear,EffectiveYearBuilt
117020,-0.535347,-0.133617,0.887178,-0.241331
102924,1.210640,0.885396,1.356438,1.642879
78262,0.729679,0.885396,0.619029,0.241800
156735,-0.490257,-0.133617,0.216806,0.048547
186136,1.654026,0.885396,0.417918,0.386739


In [16]:
def mse(theta0, theta1, theta2, theta3, theta4, X, y):
    y_hat = theta0 + theta1*X.iloc[:, 0] + theta2*X.iloc[:, 1] + theta3*X.iloc[:, 2] + theta4*X.iloc[:, 3]
    return np.mean((y_hat - y) ** 2)

def grad_mse(theta0, theta1, theta2, theta3, theta4, X, y):
    y_hat = theta0 + theta1*X.iloc[:, 0] + theta2*X.iloc[:, 1] + theta3*X.iloc[:, 2] + theta4*X.iloc[:, 3]
    n = X.shape[0]
    grad_0 = (-2 / n) * sum(y - y_hat)
    grad_1 = (-2 / n) * sum(X.iloc[:, 0] * (y - y_hat)) 
    grad_2 = (-2 / n) * sum(X.iloc[:, 1] * (y - y_hat)) 
    grad_3 = (-2 / n) * sum(X.iloc[:, 2] * (y - y_hat)) 
    grad_4 = (-2 / n) * sum(X.iloc[:, 3] * (y - y_hat)) 
    return grad_0, grad_1, grad_2, grad_3, grad_4

In [17]:
def minimize_GD(epochs, loss_fn, grad_loss_fn, X, y, alpha=0.2):
    """
    Uses gradient descent to minimize loss_fn. Returns the minimizing value of
    theta once all thetas change less than 0.001 between iterations.
    """
    
    # Set starting epoch
    epochs = np.arange(epochs)
    epochs_conv = []
    
    # Set loss array
#     losses = []
    losses_conv = []
    
    # Set theta array
#     theta0s = []
#     theta1s = []
#     theta2s = []
#     theta3s = []
#     theta4s = []
    
    # Set starting theta
    theta0 = 10
    theta1 = 10
    theta2 = 10
    theta3 = 10
    theta4 = 10
    
    for epoch in epochs:

        # Update losses
#         losses.append(loss_fn(theta0, theta1, theta2, theta3, theta4, X, y))
        
        # Update thetas
#         theta0s.append(theta0)
#         theta1s.append(theta1)
#         theta1s.append(theta2)
#         theta1s.append(theta3)
#         theta1s.append(theta4)
        
        # Compute gradient
        grad_0, grad_1, grad_2, grad_3, grad_4 = grad_loss_fn(theta0, theta1, theta2, theta3, theta4, X, y)
        
        # Get new theta
        new_theta0 = theta0 - alpha * grad_0
        new_theta1 = theta1 - alpha * grad_1
        new_theta2 = theta2 - alpha * grad_2
        new_theta3 = theta3 - alpha * grad_3
        new_theta4 = theta4 - alpha * grad_4
        
        if (
            abs(new_theta0 - theta0) < 0.001 
            and abs(new_theta1 - theta1) < 0.001
            and abs(new_theta2 - theta2) < 0.001
            and abs(new_theta3 - theta3) < 0.001
            and abs(new_theta4 - theta4) < 0.001
        ):
            epochs_conv.append(epoch)
            losses_conv.append(loss_fn(theta0, theta1, theta2, theta3, theta4, X, y))
            break
          
        # Update theta
        theta0 = new_theta0
        theta1 = new_theta1
        theta2 = new_theta2
        theta3 = new_theta3
        theta4 = new_theta4
        
    return theta0, theta1, theta2, theta3, theta4, epochs_conv, losses_conv

In [18]:
theta0, theta1, theta2, theta3, theta4, epochs_conv, losses_conv = minimize_GD(
    epochs = 1000,
    loss_fn = mse,
    grad_loss_fn = grad_mse,
    X = X_train,
    y = y_train,
    alpha = 0.2
)

print(theta0)
print(theta1)
print(theta2)
print(theta3)
print(theta4)
print(epochs_conv[0])
print(losses_conv[0]**0.5)

### Try with 2 Features

In [20]:
def process_data_GD_2D(data):
    """Combine all pipelines to create processed data."""
    # Transform Data, Select Features
    data = select_columns(data, 
                         'SQFTmain',
                          'LandBaseYear',
                         'TotalValue'
                         )
    
    # Standardize Data using Standard Scaling
    columns = ['SQFTmain', 'LandBaseYear']
    
    data = standardize_data(data, columns)
    
    # Return predictors and response variables separately
    X = data.drop(['TotalValue'], axis = 1)
    y = data.loc[:, 'TotalValue']
    
    return X, y

In [21]:
X_train, y_train = process_data_GD_2D(train)
    
X_train.head()

,SQFTmain,LandBaseYear
117020,-0.535347,0.887178
102924,1.210640,1.356438
78262,0.729679,0.619029
156735,-0.490257,0.216806
186136,1.654026,0.417918


In [22]:
def mse_2D(theta0, theta1, theta2, X, y):
    y_hat = theta0 + theta1*X.iloc[:, 0] + theta2*X.iloc[:, 1]
    return np.mean((y_hat - y) ** 2)

def grad_mse_2D(theta0, theta1, theta2, X, y):
    y_hat = theta0 + theta1*X.iloc[:, 0] + theta2*X.iloc[:, 1]
    n = X.shape[0]
    grad_0 = (-2 / n) * sum(y - y_hat)
    grad_1 = (-2 / n) * sum(X.iloc[:, 0] * (y - y_hat)) 
    grad_2 = (-2 / n) * sum(X.iloc[:, 1] * (y - y_hat)) 
    
    return grad_0, grad_1, grad_2

In [23]:
def minimize_GD_2D(epochs, loss_fn, grad_loss_fn, X, y, alpha=0.2):
    """
    Uses gradient descent to minimize loss_fn. Returns the minimizing value of
    theta once all thetas change less than 0.001 between iterations.
    """
    
    # Set starting epoch
    epochs = np.arange(epochs)
    epochs_conv = []
    
    # Set loss array
#     losses = []
    losses_conv = []
    
    # Set theta array
#     theta0s = []
#     theta1s = []
#     theta2s = []
    
    # Set starting theta
    theta0 = 10
    theta1 = 10
    theta2 = 10
    
    for epoch in epochs:

        # Update losses
#         losses.append(loss_fn(theta0, theta1, theta2, X, y))
        
        # Update thetas
#         theta0s.append(theta0)
#         theta1s.append(theta1)
#         theta2s.append(theta2)
        
        # Compute gradient
        grad_0, grad_1, grad_2 = grad_loss_fn(theta0, theta1, theta2, X, y)
        
        # Get new theta
        new_theta0 = theta0 - alpha * grad_0
        new_theta1 = theta1 - alpha * grad_1
        new_theta2 = theta2 - alpha * grad_2
        
        if (
            abs(new_theta0 - theta0) < 0.001 
            and abs(new_theta1 - theta1) < 0.001
            and abs(new_theta2 - theta2) < 0.001
        ):
            epochs_conv.append(epoch)
            losses_conv.append(loss_fn(theta0, theta1, theta2, X, y))
            break
          
        # Update theta
        theta0 = new_theta0
        theta1 = new_theta1
        theta2 = new_theta2
        
    return theta0, theta1, theta2, epochs_conv, losses_conv

In [24]:
theta0, theta1, theta2, epochs_conv, losses_conv = minimize_GD_2D(
    epochs = 100,
    loss_fn = mse_2D,
    grad_loss_fn = grad_mse_2D,
    X = X_train,
    y = y_train,
    alpha = 0.2
)

print(theta0)
print(theta1)
print(theta2)
print(epochs_conv[0])
print(losses_conv[0]**0.5)

504144.5765834486
199984.84510353935
228096.07552135552
38
251137.09969501154
